<a href="https://colab.research.google.com/github/tvut/mais/blob/main/Garbage_Collection_Model_Final_Results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import requirements.

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import datasets, layers, models

Install kaggle and setup my config file. This will probably be need to run twice so the directory is created.

In [ ]:
api_token = {"username":"tristanssdfasdf","key":"60aac443aa53780276ab45e3ece8082f"}
import json
import zipfile
import os
!kaggle

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 /root/.kaggle/kaggle.json

usage: kaggle [-h] [-v] {competitions,c,datasets,d,kernels,k,config} ...
kaggle: error: the following arguments are required: command


Use kaggle to download the dataset.

In [ ]:
!kaggle config path -p /content
!kaggle datasets download -d mostafaabla/garbage-classification
!unzip /content/garbage-classification.zip

usage: kaggle config [-h] {view,set,unset} ...
kaggle config: error: argument command: invalid choice: 'path' (choose from 'view', 'set', 'unset')
garbage-classification.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  /content/garbage-classification.zip
replace garbage_classification/battery/battery1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

Checks that our data is actually here. Data length is known and we open an image just to make sure its there.

In [ ]:
import os,glob

dir_path = 'garbage_classification'
img_list = glob.glob(os.path.join(dir_path, '*/*.jpg'))
len(img_list)

print(f"{len(img_list)} images found.")
assert (len(img_list) == 15515)

import PIL
PIL.Image.open(str(img_list[0]))


Importing the data using the built in dataset from directory generator as specified in the [docs](https://www.tensorflow.org/tutorials/load_data/images#:~:text=Note%3A%20You%20previously%20resized%20images%20using%20the%20image_size,the%20tf.keras.layers.Resizing%20layer.%20Configure%20the%20dataset%20for%20performance).

This resizes our data and handles the validation data split. Note that seeds need to be the same for the validation set to be the same.

In [ ]:
td64 = tf.keras.utils.image_dataset_from_directory(
    dir_path,
    labels='inferred',
    label_mode='int',
    class_names=None,
    color_mode='rgb',
    batch_size=256,
    image_size=(64, 64),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset="training",
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False
)

print()

vd64 = tf.keras.utils.image_dataset_from_directory(
    dir_path,
    labels='inferred',
    label_mode='int',
    class_names=None,
    color_mode='rgb',
    batch_size=256,
    image_size=(64, 64),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset="validation",
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False
)
import matplotlib.pyplot as plt

print("\nClasses:")
plt.figure(figsize=(10, 10))
for images, labels in td64.take(1):
  for i in range(len(td64.class_names)):
    ax = plt.subplot(3, 4, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(td64.class_names[labels[i]])
    plt.axis("off")

In [ ]:
td128 = tf.keras.utils.image_dataset_from_directory(
    dir_path,
    labels='inferred',
    label_mode='int',
    class_names=None,
    color_mode='rgb',
    batch_size=256,
    image_size=(128, 128),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset="training",
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False
)

print()

vd128 = tf.keras.utils.image_dataset_from_directory(
    dir_path,
    labels='inferred',
    label_mode='int',
    class_names=None,
    color_mode='rgb',
    batch_size=256,
    image_size=(128, 128),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset="validation",
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False
)
import matplotlib.pyplot as plt

print("\nClasses:")
plt.figure(figsize=(10, 10))
for images, labels in td128.take(1):
  for i in range(len(td128.class_names)):
    ax = plt.subplot(3, 4, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(td128.class_names[labels[i]])
    plt.axis("off")

Model was taken from the [tensorflow tutorial page](https://www.tensorflow.org/tutorials/images/cnn)

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.summary()

In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(12))

model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


history = model.fit(td128, batch_size=32, epochs=10, validation_data=(vd))

In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model


class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(12)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

model = MyModel()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)



@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

EPOCHS = 10

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in td:
    train_step(images, labels)

  for test_images, test_labels in vd:
    test_step(test_images, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )


Resnet50 pretrained weights with 64,64 images

In [ ]:
from keras.applications.resnet import ResNet50

model = models.Sequential()
model.add(ResNet50(include_top=False, weights='imagenet', input_shape=(64, 64, 3), pooling='avg'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(12))

model.summary()


model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


history = model.fit(td64, batch_size=32, epochs=19, validation_data=(vd64))

Resnet50 pretrained weights with 128,128 images

In [ ]:
from keras.applications.resnet import ResNet50

model = models.Sequential()
model.add(ResNet50(include_top=False, weights='imagenet', input_shape=(128, 128, 3), pooling='avg'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(12))

model.summary()


model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


history = model.fit(td128, batch_size=32, epochs=14, validation_data=(vd128))

Resnet50 pretrained weights with 128,128 images
Try 2

In [ ]:
from keras.applications.resnet import ResNet50

model2 = models.Sequential()
model2.add(ResNet50(include_top=False, weights='imagenet', input_shape=(128, 128, 3), pooling='avg'))
model2.add(layers.Flatten())
model2.add(layers.Dense(64, activation='relu'))
model2.add(layers.Dense(12))

model2.summary()


model2.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


history = model2.fit(td128, batch_size=32, epochs=15, validation_data=(vd128))

Saving the model of try 2

In [ ]:
model2.save('model.txt')
model2 = keras.models.load_model('model.txt')

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(td.labels, model2.prediction(td))

In [ ]:
from google.colab import drive
import pandas as pd
import cv2 as cv
from PIL import Image

drive.mount('/content/drive')
img = cv.imread('/content/drive/MyDrive/Garbage_photos/cardboard.jpg')
img = img.resize((128,128))
img.display()
#print(image)

model2.predict(img)

In [ ]:
tf.test.gpu_device_name()

In [ ]:
!pip install transformers "datasets>=1.17.0" tensorboard --upgrade

from huggingface_hub import notebook_login

notebook_login()

model_id = "google/vit-base-patch16-224-in21k"

from huggingface_hub import HfFolder
import tensorflow as tf

# id2label = {str(i): label for i, label in enumerate(img_class_labels)}
# label2id = {v: k for k, v in id2label.items()}

num_train_epochs = 5
train_batch_size = 32
eval_batch_size = 32
learning_rate = 3e-5
weight_decay_rate=0.01
num_warmup_steps=0
output_dir=model_id.split("/")[1]
hub_token = HfFolder.get_token() # or your token directly "hf_xxx"
hub_model_id = f'{model_id.split("/")[1]}-euroSat'
fp16=True

# Train in mixed-precision float16
# Comment this line out if you're using a GPU that will not benefit from this
if fp16:
  tf.keras.mixed_precision.set_global_policy("mixed_float16")

from transformers import TFViTForImageClassification, create_optimizer

# create optimizer wight weigh decay
num_train_steps = len(tf_train_dataset) * num_train_epochs
optimizer, lr_schedule = create_optimizer(
    init_lr=learning_rate,
    num_train_steps=num_train_steps,
    weight_decay_rate=weight_decay_rate,
    num_warmup_steps=num_warmup_steps,
)

# load pre-trained ViT model
model = TFViTForImageClassification.from_pretrained(
    model_id,
    num_labels=len(img_class_labels),
    id2label=id2label,
    label2id=label2id,
)

# define loss
loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# define metrics
metrics=[
    tf.keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
    tf.keras.metrics.SparseTopKCategoricalAccuracy(3, name="top-3-accuracy"),
]

# compile model
model.compile(optimizer=optimizer,
              loss=loss,
              metrics=metrics
              )

train_results = model.fit(
    td,
    validation_data=vd,
    callbacks=callbacks,
    epochs=num_train_epochs,
)